real_estate 테이블에 부동산 값 넣기

In [1]:
import pandas as pd

df = pd.read_csv('../../data/rent.csv', encoding='EUC-KR')

df

,접수연도,자치구코드,자치구명,법정동코드,법정동명,지번구분,지번구분명,본번,부번,층,...,보증금(만원),임대료(만원),건물명,건축년도,건물용도,계약기간,신규갱신여부,계약갱신권사용여부,종전 보증금,종전 임대료
0,2025,11290,성북구,12100,안암동1가,1.0,대지,10.0,1.0,4.0,...,14000,0,(10-1),2013.0,연립다세대,25.05~27.04,갱신,NaN,14000.0,NaN
1,2025,11170,용산구,10100,후암동,NaN,NaN,NaN,NaN,NaN,...,4000,0,NaN,1966.0,단독다가구,25.03~27.03,신규,NaN,NaN,NaN
2,2025,11410,서대문구,10700,옥천동,1.0,대지,126.0,171.0,3.0,...,25000,0,(126-171),2019.0,연립다세대,25.05~27.05,신규,NaN,NaN,NaN
3,2025,11380,은평구,10900,신사동,NaN,NaN,NaN,NaN,NaN,...,25000,0,NaN,1971.0,단독다가구,25.04~27.04,신규,NaN,NaN,NaN
4,2025,11740,강동구,10700,암사동,1.0,대지,485.0,31.0,5.0,...,6562,7,(485-31),2020.0,연립다세대,25.03~27.03,갱신,NaN,6540.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137794,2025,11710,송파구,10800,문정동,1.0,대지,1.0,0.0,3.0,...,75000,180,문정래미안,2004.0,아파트,24.12~26.12,신규,NaN,NaN,NaN
137795,2025,11290,성북구,10100,성북동,1.0,대지,166.0,1.0,4.0,...,2000,90,파크리움,2017.0,연립다세대,25.01~27.01,신규,NaN,NaN,NaN
137796,2025,11230,동대문구,10500,답십리동,1.0,대지,497.0,23.0,4.0,...,5000,100,청계푸르지오시티,2015.0,아파트,24.12~26.12,신규,NaN,NaN,NaN
137797,2025,11740,강동구,10700,암사동,1.0,대지,516.0,19.0,3.0,...,31500,0,"휴먼리버빌(A,B,C동)",2012.0,연립다세대,24.12~26.12,갱신,NaN,31500.0,NaN


In [2]:
df = df[['자치구명', '법정동명', '전월세 구분', '임대면적', '보증금(만원)', '임대료(만원)' ]]
df = df.drop_duplicates()
df.columns = ['gu', 'dong', 'rent_type', 'size', 'rent_deposit','rent_fee']

In [3]:
df

,gu,dong,rent_type,size,rent_deposit,rent_fee
0,성북구,안암동1가,전세,19.80,14000,0
1,용산구,후암동,전세,45.62,4000,0
2,서대문구,옥천동,전세,37.94,25000,0
3,은평구,신사동,전세,65.55,25000,0
4,강동구,암사동,월세,35.36,6562,7
...,...,...,...,...,...,...
137794,송파구,문정동,월세,170.70,75000,180
137795,성북구,성북동,월세,29.72,2000,90
137796,동대문구,답십리동,월세,30.42,5000,100
137797,강동구,암사동,전세,70.77,31500,0


전월세 평균값 구하기

In [9]:
average_rent_by_gu = df.groupby(['gu', 'dong', 'rent_type']).agg({
    'size': 'mean',             # 면적의 평균
    'rent_deposit': 'mean',     # 보증금의 평균
    'rent_fee': 'mean',         # 전월세 평균
}).reset_index()

average_rent_by_gu


,gu,dong,rent_type,size,rent_deposit,rent_fee
0,강남구,개포동,월세,63.402055,35493.748752,156.826955
1,강남구,개포동,전세,65.486649,73743.261426,0.000000
2,강남구,논현동,월세,43.351085,10508.579051,112.405138
3,강남구,논현동,전세,54.716281,46741.083333,0.000000
4,강남구,대치동,월세,61.493245,32391.832095,132.838039
...,...,...,...,...,...,...
759,중랑구,상봉동,전세,46.907424,26960.652381,0.000000
760,중랑구,신내동,월세,47.230787,6466.246445,44.289100
761,중랑구,신내동,전세,67.546599,31843.062147,0.000000
762,중랑구,중화동,월세,46.380327,5656.775701,50.154206


In [18]:
import mysql.connector

conn = mysql.connector.connect(
    host = "database-1.cbwuowec22t2.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = "team2",
    password = "1234",
    database = "eda_project"
)

In [19]:
cur = conn.cursor(buffered=True)

In [21]:
rent_list = [(row['gu'], row['dong'], row['rent_type'], row['size'],row['rent_deposit'],row['rent_fee']) for _, row in average_rent_by_gu.iterrows()]

In [22]:
len(rent_list)

764

In [23]:
def insert_rent(gu, dong, estate_type, estate_size, estate_deposit, estate_rent):
    # 구 ID 가져오기
    cur.execute('SELECT gu_id FROM gu WHERE gu_name = %s', (gu,))
    gu_id = cur.fetchone()[0]

    #동 ID 가져오기
    cur.execute('SELECT dong_id FROM dong WHERE dong_name = %s', (dong,))
    dong_id = cur.fetchone()[0]

    cur.execute(
            'INSERT INTO real_estate (gu_id, dong_id, estate_type, estate_size, estate_rent, estate_deposit) '
            'VALUES (%s, %s, %s, %s, %s, %s)', 
            (gu_id, dong_id, estate_type, estate_size, estate_rent, estate_deposit))
    
    conn.commit()

In [24]:
for gu, dong, estate_type, estate_size, estate_deposit, estate_rent in rent_list:
    insert_rent(gu, dong, estate_type, estate_size, estate_deposit, estate_rent)